In [3]:
"""
This module will run Jij calculation step from converged combined_imp_wc.
"""
from __future__ import print_function, absolute_import
from aiida.orm import Code, load_node, CalcJobNode, Float, Int, Str, Dict, SinglefileData, ArrayData
from aiida.plugins import DataFactory
from aiida.engine import if_, ToContext, WorkChain, calcfunction
from aiida.common import LinkType
from aiida.common.folders import SandboxFolder
from aiida_kkr.workflows.gf_writeout import kkr_flex_wc
from aiida_kkr.workflows.kkr_imp_sub import kkr_imp_sub_wc
from aiida_kkr.workflows.dos import kkr_dos_wc
from aiida_kkr.calculations import KkrimpCalculation
import tarfile
import os
from six.moves import range
from aiida_kkr.tools.save_output_nodes import create_out_dict_node


In [4]:
__copyright__ = (u"Copyright (c), 2021, Forschungszentrum Jülich GmbH, "
                 "IAS-1/PGI-1, Germany. All rights reserved.")
__license__ = "MIT license, see LICENSE.txt file"
__version__ = "0.0.1"
__contributors__ = (u"Rubel Mozumder", u"Philipp Rüßmann")


In [ ]:
class combined_imps_jij_wc(WorkChain):
    """
    TODO: ADD WF MECHANISM, WF INPUT PARAMS, OUTPUTS
    """
    _wf_version = __version__
    _wf_label = 'combined_imps_jij_wc'
    _wf_description = ''
    _wf_options_default = {}
    _wf_default = {'jij_run': True
                    
                  }
    
    @classmethod
    def get_wf_defaults(cls, silent=False):
        """
        Print and return _wf_defaults dictionary. Can be used easily to reset
        wf_parameters.
        
        returns _wf_defaults
        """
        if not silent:
            print('Version of workflow: {}'.format(cls._wf_version))
        
        return cls._wf_default
    @classmethod
    def define(cls, spec):
        """
        Define the features of the workflow
        
        WF PLANE:
            * Run the Jij calculation from the successful combine_imps_wc
                NEED:
                * combine_imps_wc
            * Or Run Jij calcualtion from the successful single imp workchain i.e. kkr_imp_sub_wc, and kkr_flex_wc
                NEED:
                * kkr_imp_sub_wc
                * impurity1_output_node
                * impurity2_output_node
                
        """
        # Such for define method on WorkChain and extend it as follows
        super(combined_imps_jij_wc, cls).define(spec)
         
        # Run Jij calcualtion from the successful single imp workchain i.e. kkr_imp_sub_wc, and kkr_flex_wc    
        # Inputs
        spec.expose_inputs(kkr_imp_sub_wc, 
                           namespace='scf', 
                           include=('kkrimp', 'options', 'wf_parameters',)
                           namespace_options={'required': False}
                          )
        spec.expose_inputs(kkr_flex_wc,
                           namespace='host_gf',
                           include=('kkr', 'options', 'params_kkr_overwrite',), # expose only those port which are not set automatically
                           namespace_options={'required': False}, # this makes sure the kkr code input is not needed if gf_host_remote is provided and the entire namespace is omitted
                          )
        spec.input("impurity1_output_node", required=False, valid_type=Dict, #TODO make validator for input node to make sure it is the output of kkr_imp_wc
                   help="""
Output node of a single impurity calculation. This can be the output of either the `kkr_imp_wc`, `kkr_imp_sub_wc`
workflows or of an `KkrimpCalculation`.
Use these output Dict nodes:
  * for `kkr_imp_wc`: single_imp_worlfow.outputs.workflow_info
  * for `kkr_imp_sub_wc`: single_imp_worlfow.outputs.workflow_info
  * for `KkrimpCalculation`: single_imp_worlfow.outputs.output_parameters
""")
        spec.input("impurity2_output_node", required=False, valid_type=Dict,
                   help="Output node of second single impurity calculation. See help string of `impurity1_output_node` for more details.")
        
        spec.input("offset_imp2", valid_type=Dict, required=False,
                   help="""Offset of the secon impurity with respect to the first impurity.
Can be given either via the 'vector' or the 'index' keys in the dictionary.
The 'vector' option allows to give the offset vector in cartesian units and 
the 'index' option allows to five the offset vector in units of the lattice 
vectors of the host system's structure.""")
        
        spec.input("gf_host_remote", valid_type=RemoteData, required=False, #TODO add validator that makes sure this is not given together with the host_gf sub-workflow namespace
                   help="""RemoteData node of pre-calculated host Green function (i.e. with kkr_flex_wc).
If given then the writeout step of the host GF is omitted.""")
        
        # Run the Jij calculation from the successful combine_imps_wc
        # Inputs
        spec.expose_inputs(combine_imps_wc, 
                           namespace='scf_combined', 
                           include=('kkrimp', 'options', 'wf_parameters',)
                           namespace_options={'required': False}
                          )
        spec.input("impurity_conbined_output_node", required=False, valid_type=Dict, #TODO make validator for input node to make sure it is the output of kkr_imp_wc
                   help="""
Output node from combine_imps_wc. The output either from `combine_imps_wc` or its `kkr_imp_sub_wc` workflow or its 
last `KkrimpCalculation`.
Use these output Dict nodes:
  * for `combine_imp_wc`: WF_NODE.outputs.workflow_info
  * for `kkr_imp_sub_wc`: WF_NODE.outputs.workflow_info
  * for `KkrimpCalculation`: CALC_NODE.outputs.output_parameters
""")

    # Outputs
    spec.output('workflow_info', valid_type= Dict)
    # Beside the conventional keys as other 'workflows_info' contains, it should contain
    #  1. host_imp_pot (uuid) from the coverge_imp_cluster 
    spec.output('JijData' valid_type=ArrayData)
        